# 2019 MolSSI Summer School QM project: semiempirical model of Argon

## 1. Introduction

In this project, we will simulate a cluster of Argon atoms using quantum mechanics (QM) to calculate their total energy. First-principles QM simulations are complicated and expensive, and a quick implementation would rely on a substantial amount of pre-existing software infrastructure. Instead, we will implement a much simpler semiempirical QM simulation that has been designed and parameterized to reproduce first-principles QM data using a minimal model. We can then limit our external dependencies to the standard numerical functionality of Python:

In [33]:
import numpy as np

As is typically the case in quantum chemistry, we will treat the atomi nuclei as classical point charges,

$$ \vec{r}_i = (x_i , y_i, z_i). $$

While we would normally read these from a file or pass them in as an argument to a function, we will just specify a set of atomic coordinates in this example:

In [2]:
atomic_coordinates = np.array([ [0.0,0.0,0.0], [3.0,4.0,5.0] ])
number_of_atoms = len(atomic_coordinates)

np.set_printoptions(precision=1)
print(number_of_atoms)
print(atomic_coordinates)

2
[[0. 0. 0.]
 [3. 4. 5.]]


Similarly, we will only calculate and print the total energy within this notebook, whereas more useful software would probably print output data to a file or return it from a function call. You will add such features as you refactor this software. All physical quantities in this project will be specified in Hartree atomic units, where the bohr is the unit of length and the hartree is the unit of energy.

## 2. Model Hamiltonian

As is standard in quantum chemistry, we will assume that the total energy of our system is defined to be the ground state energy of a quantum many-body Hamiltonian $\hat{H}$. In second quantization notation, we can write it as

$$ \hat{H} = E_{\mathrm{ion}} + \sum_{p,q} \sum_{\sigma \in \{ \uparrow , \downarrow \} } h_{p,q} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{q,\sigma} + \tfrac{1}{2}\sum_{p,q,r,s} \sum_{\sigma,\sigma' \in \{ \uparrow , \downarrow \} } V_{p,q,r,s} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{r,\sigma'}^{\dagger} \hat{a}_{s,\sigma'} \hat{a}_{q,\sigma} , $$

where $\hat{a}_{p,\sigma}^{\dagger}$ and $\hat{a}_{p,\sigma}$ are the electron raising and lowering operators for an atomic orbital index $p$ and spin $\sigma$. We will not be using $\hat{H}$ itself in our calculations, but we will make use of the coefficient tensors $h_{p,q}$ and $V_{p,q,r,s}$. In first-principles calculations, each element of $h_{p,q}$ and $V_{p,q,r,s}$ would require the evaluation of a complicated integral. In our semiempirical model, we will set most of them to zero and assign a simple analytical form to the rest of them. The notation being used here is mostly consistent with modern quantum chemistry notation, but some objects, particularly $V_{p,q,r,s}$, have multiple conventions in practice.

### A. Model design & parameters

This semiempirical model combines some standard concepts and methods used in physics and chemistry. First, it will use a minimal number of electronic degrees of freedom. Because Argon is a noble gas, it interacts primarily through London dispersion forces that are mediated by quantum dipole fluctuations. The lowest energy dipole transition is from the occupied $3p$ states to the unoccupied $4s$ state, and we will include these 4 atomic orbitals per atom. Similarly, we will use a multipole expansion to simplify electronic excitations and retain only the monopole and dipole terms, which also restricts electronic polarization to 4 degrees of freedom per atom. We will use $\{s, p_x, p_y, p_z\}$ to label both atomic orbitals and multipole moments on each atom. The nuclear charge of Argon is 18, but our model combines the nucleus and the 12 neglected electrons ($1s^2$, $2s^2$, $2p^6$, and $3s^2$) into an ionic point charge with $Z = 6$.

In [3]:
ionic_charge = 6
orbital_types = ['s', 'px' ,'py', 'pz']
orbitals_per_atom = len(orbital_types)

p_orbitals = orbital_types[1:]
print(p_orbitals)

['px', 'py', 'pz']


The atomic orbital index contains information about which atom the orbital is centered on and the type of orbital it is. We will often extract these individual pieces of information using $\mathrm{atom}(p)$ to denote the atom's index and $\mathrm{orb}(p)$ to denote the orbital type. This is the first of many instances in this project where we could either represent something as a pre-tabulate list or a function. We will always make the simpler choice, in this case functions:

In [4]:
def atom(ao_index):
    '''Returns the atom index part of an atomic orbital index.'''
    return ao_index // orbitals_per_atom

def orb(ao_index):
    '''Returns the orbital type of an atomic orbital index.'''
    orb_index = ao_index % orbitals_per_atom
    return orbital_types[orb_index]

def ao_index(atom_p,orb_p):
    '''Returns the atomic orbital index for a given atom index and orbital type.'''
    p = atom_p*orbitals_per_atom
    p += orbital_types.index(orb_p)
    return p

print("1st atom index =", atom(0))
print("1st orbital type =", orb(0))
print("ao index of s orbital on 1st atom =", ao_index(0, 's'))

1st atom index = 0
1st orbital type = s
ao index of s orbital on 1st atom = 0


We will discuss the model parameters in more detail as they are used, but it is a good idea to collect them all in a common data structure, a Python dictionary, for convenient access throughout the notebook:

In [36]:
model_parameters = { 'r_hop' : 5.0, # hopping length scale
                     't_ss' : -0.002, # s-s hopping energy scale
                     't_sp' : -0.004, # s-p hopping energy scale
                     't_pp1' : -0.006, # 1st p-p hopping energy scale
                     't_pp2' : -0.008, # 2nd p-p hopping energy scale
                     'r_pseudo' : 6.0, # pseudopotential length scale
                     'v_pseudo' : 0.5, # pseudopotential energy scale
                     'dipole' : 4.0, # dipole strength of s-p transition
                     'energy_s' : 1.3, # onsite energy of s orbital
                     'energy_p' : 0.1, # onsite energy of p orbital
                     'self_energy' : 0.5 } # Coulomb self-energy of all orbitals

There are no parameters related to orbital overlap because all atomic orbitals are assumed to be orthogonal. The parameter values have been pre-optimized for this project, but the fitting process and reference data are both listed at the end of the project if you'd like to learn more about them.

### B. Slater-Koster tight-binding model

We will describe the kinetic energy of electrons using a simplified [Slater-Koster tight-binding method](https://en.wikipedia.org/wiki/Tight_binding). Because of the symmetry of atomic orbitals and the translational invariance of the kinetic energy operator, there are 4 distinct, distance-dependent "hopping" energies that characterize the interatomic kinetic energy between s and p orbitals:

![s-p hopping diagram](hopping_cases.png)

All other atomic orientations can be related to these cases by a change of coordinates. While it is compatible with very general functional forms, we will use a Gaussian form to simplify the model and its implementation. The distance-dependence of this simple version is controlled by a single hopping length scale $r_{\mathrm{hop}}$ and the strength of each type of hopping energy,

$$ t_{o,o'}(\vec{r}) = \exp(1-r^2/r_{\mathrm{hop}}^2) \times \begin{cases}
 t_{ss} , & o = o' = s \\
 [\vec{o}' \cdot (\vec{r}/r_{\mathrm{hop}})] t_{sp}, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -[\vec{o} \cdot (\vec{r}/r_{\mathrm{hop}})] t_{sp} , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (r^2/r_{\mathrm{SK}}^2)\,(\vec{o} \cdot \vec{o}')  t_{pp2} - [\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] (t_{pp1} + t_{pp2}), & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$
 
where $o$ and $o'$ are the orbital types of the 1st and 2nd atoms and $\vec{r}$ is a vector pointing from the 2nd atom to the 1st atom. We are assigning direction vectors to the p orbitals, $\vec{p}_x \equiv (1,0,0)$, $\vec{p}_y \equiv (0,1,0)$, and $\vec{p}_z \equiv (0,0,1)$, to simplify the notation. This project has multiple case-based formulas, and we will implement them using a code structure similar to each formula:

In [40]:
vec = { 'px':[1,0,0], 'py':[0,1,0], 'pz':[0,0,1] }

def hopping_energy(o1, o2, r12, model_parameters):
    '''Returns the hopping matrix element for a pair of orbitals of type o1 & o2 separated by a vector r12.'''
    r12_rescaled = r12 / model_parameters['r_hop']
    r12_squared = np.dot(r12_rescaled, r12_rescaled)
    ans = np.exp( 1.0 - r12_squared )
    if o1 == 's' and o2 == 's':
        ans *= model_parameters['t_ss']
    if o1 == 's' and o2 in p_orbitals:
        ans *= np.dot(vec[o2], r12_rescaled) * model_parameters['t_sp']
    if o2 == 's' and o1 in p_orbitals:
        ans *= -np.dot(vec[o1], r12_rescaled)* model_parameters['t_sp']
    if o1 in p_orbitals and o2 in p_orbitals:
        ans *= ( r12_squared * np.dot(vec[o1], vec[o2]) * model_parameters['t_pp2']
                 - np.dot(vec[o1], r12_rescaled) * np.dot(vec[o2], r12_rescaled)
                 * ( model_parameters['t_pp1'] + model_parameters['t_pp2'] ) )
    return ans

print("vec[px] =",vec['px'])
print("hopping test",hopping_energy('s','px',np.array([5.0,0.0,0.0]),model_parameters))

vec[px] = [1, 0, 0]
hopping test -0.004


### C. Coulomb interaction

For the purpose of electrostatics, we will describe all inter-atomic Coulomb interactions with point charges and all intra-atomic Coulomb interactions between electrons with a common electronic self-energy parameter $V_0$. We need access to both the interaction kernel and its derivatives to define the multipole expansion,

$$ V_{o,o'}(\vec{r}) = \begin{cases}
 1/r , & o = o' = s \\
 (\vec{o}' \cdot \vec{r}) / r^3, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -(\vec{o} \cdot \vec{r}) / r^3 , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (\vec{o} \cdot \vec{o}') / r^3 - 3 (\vec{o} \cdot \vec{r}) (\vec{o}' \cdot \vec{r}) / r^5, & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$

In [44]:
def coulomb_energy(o1, o2, r12):
    '''Returns the Coulomb matrix element for a pair of multipoles of type o1 & o2 separated by a vector r12.'''
    r12_length = np.linalg.norm(r12)
    if o1 == 's' and o2 == 's':
        ans = 1.0 / r12_length
    if o1 == 's' and o2 in p_orbitals:
        ans = np.dot(vec[o2], r12) / r12_length**3
    if o2 == 's' and o1 in p_orbitals:
        ans = -np.dot(vec[o1], r12) / r12_length**3
    if o1 in p_orbitals and o2 in p_orbitals:
        ans = ( np.dot(vec[o1], vec[o2]) / r12_length**3
               - 3.0 * np.dot(vec[o1], r12) * np.dot(vec[o2], r12) / r12_length**5 )
    return ans 

The semiempirical model approximations strongly distort the physics of inter-atomic Pauli repulsion, and we compensate for these errors with a short-range ionic pseudopotential, which is a common tool in physics for building effective models of ionic cores:

$$ V_o^{\mathrm{pseudo}}(\vec{r}) = V_{\mathrm{pseudo}} \exp(1 - r^2/r_{\mathrm{pseudo}}^2) \times \begin{cases}
1, & o = s \\
2 \vec{o}\cdot\vec{r}/r_{\mathrm{pseudo}}, & o \in \{p_x, p_y, p_z\}. \end{cases} $$

In [42]:
def pseudopotential_energy(o, r, model_parameters):
    '''Returns the energy of a pseudopotential between a multipole of type o and an atom separated by a vector r.'''
    ans = model_parameters['v_pseudo']
    r_rescaled = r / model_parameters['r_pseudo']
    ans *= np.exp( 1.0 - np.dot(r_rescaled,r_rescaled) )
    if o in p_orbitals:
        ans *= -2.0 * np.dot(vec[o], r_rescaled)
    return ans

These interaction kernels enable us to define and calculate the ion-ion energy in $\hat{H}$,

$$ E_{\mathrm{ion}} = Z^2 \sum_{i < j} V_{s,s}(\vec{r}_i - \vec{r}_j) $$

In [46]:
def calculate_energy_ion(atomic_coordinates):
    '''Returns the ionic contribution to the total energy for an input list of atomic coordinates.'''
    energy_ion = 0.0
    for i, r_i in enumerate(atomic_coordinates):
        for j, r_j in enumerate(atomic_coordinates):
            if i < j:
                energy_ion += (ionic_charge**2)*coulomb_energy('s', 's', r_i - r_j)
    return energy_ion

energy_ion = calculate_energy_ion(atomic_coordinates)
print(energy_ion)

5.091168824543142


the vector of electron-ion interactions,

$$ V^{\mathrm{ion}}_p = \sum_{i \neq \mathrm{atom}(p)}
   V_{\mathrm{orb}(p)}^{\mathrm{pseudo}}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i)
   - Z V_{\mathrm{orb}(p),s}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i)$$

In [76]:
def calculate_potential_vector(atomic_coordinates, model_parameters):
    '''Returns the electron-ion potential energy vector for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    potential_vector = np.zeros(ndof)
    for p in range(ndof):
        potential_vector[p] = 0.0
        for atom_i,r_i in enumerate(atomic_coordinates):
            r_pi = atomic_coordinates[atom(p)] - r_i
            if atom_i != atom(p):
                potential_vector[p] += ( pseudopotential_energy(orb(p), r_pi, model_parameters) 
                                         - ionic_charge * coulomb_energy(orb(p), 's', r_pi) )
    return potential_vector

potential_vector = calculate_potential_vector(atomic_coordinates, model_parameters)
print(potential_vector)

[-0.509623  0.287993  0.383991  0.479989 -0.509623 -0.287993 -0.383991
 -0.479989]


and the matrix of electron-electron interaction matrix elements,

$$ V^{\mathrm{ee}}_{p,q} = \begin{cases}
 V_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)}) , & \mathrm{atom}(p) \neq \mathrm{atom}(q) \\
 V_0 \delta_{p,q} , & \mathrm{atom}(p) = \mathrm{atom}(q)
\end{cases} . $$

In [51]:
def calculate_interaction_matrix(atomic_coordinates, model_parameters):
    '''Returns the electron-electron interaction energy matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    interaction_matrix = np.zeros( (ndof,ndof) )
    for p in range(ndof):
        for q in range(ndof):
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                interaction_matrix[p,q] = coulomb_energy(orb(p), orb(q), r_pq)
            if p == q:
                interaction_matrix[p,q] = model_parameters['self_energy']
    return interaction_matrix

interaction_matrix = calculate_interaction_matrix(atomic_coordinates, model_parameters)
print(interaction_matrix)

[[ 5.000000e-01  0.000000e+00  0.000000e+00  0.000000e+00  1.414214e-01
  -8.485281e-03 -1.131371e-02 -1.414214e-02]
 [ 0.000000e+00  5.000000e-01  0.000000e+00  0.000000e+00  8.485281e-03
   1.301076e-03 -2.036468e-03 -2.545584e-03]
 [ 0.000000e+00  0.000000e+00  5.000000e-01  0.000000e+00  1.131371e-02
  -2.036468e-03  1.131371e-04 -3.394113e-03]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  5.000000e-01  1.414214e-02
  -2.545584e-03 -3.394113e-03 -1.414214e-03]
 [ 1.414214e-01  8.485281e-03  1.131371e-02  1.414214e-02  5.000000e-01
   0.000000e+00  0.000000e+00  0.000000e+00]
 [-8.485281e-03  1.301076e-03 -2.036468e-03 -2.545584e-03  0.000000e+00
   5.000000e-01  0.000000e+00  0.000000e+00]
 [-1.131371e-02 -2.036468e-03  1.131371e-04 -3.394113e-03  0.000000e+00
   0.000000e+00  5.000000e-01  0.000000e+00]
 [-1.414214e-02 -2.545584e-03 -3.394113e-03 -1.414214e-03  0.000000e+00
   0.000000e+00  0.000000e+00  5.000000e-01]]


### D. Multipole decomposition

To define $V_{p,q,r,s}$ based on the Coulomb matrix elements $V_{p,q}^{\mathrm{ee}}$, we need to define a mapping from products of atomic orbitals to a linear combination of terms in the multipole expansion of electronic charge. Because the atomic orbitals are normalized, their monopole coefficient with themselves is 1 (i.e. they have unit charge). Because of orthogonality, there is no monopole term for either intra-atomic or inter-atomic transitions between atomic orbitals. We will ignore dipole transitions between atoms, which corresponds to the [neglect of diatomic differential overlap](https://en.wikipedia.org/wiki/NDDO) (NDDO) approximation that is commonly used in semiempirical quantum chemistry. All that remains is the intra-atomic s-p transition, and we will use a model parameter, $D$, to define its dipole strength. These transformation rules between atomic orbitals and multipole moments can be summarized pictorially,

![s-p multipole diagram](multipole_cases.png)

or mathematically as a 3-index tensor $\chi_{p,q,r}$ where $p$ and $q$ are the atomic orbital indices and $r$ is the multipole moment index,

$$ \chi_{p, q, r} = \begin{cases} 1, & \mathrm{orb}(p) = \mathrm{orb}(q) \ \& \ \mathrm{orb}(r) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\ 
 D , & \mathrm{orb}(q) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(p) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 D , & \mathrm{orb}(p) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(q) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 0, & \mathrm{otherwise} \end{cases} . $$

In [53]:
def chi_on_atom(o1, o2, o3, model_parameters):
    '''Returns the value of the chi tensor for 3 orbital indices on the same atom.'''
    if o1 == o2 and o3 == 's':
        return 1.0
    if o1 == o3 and o3 in p_orbitals and o2 == 's':
        return model_parameters['dipole']
    if o2 == o3 and o3 in p_orbitals and o1 == 's':
        return model_parameters['dipole']
    return 0.0

def calculate_chi_tensor(atomic_coordinates, model_parameters):
    '''Returns the chi tensor for an input list of atomic coordinates'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    chi_tensor = np.zeros( (ndof,ndof,ndof) )
    for p in range(ndof):
        for orb_q in orbital_types:
            q = ao_index(atom(p), orb_q) # p & q on same atom
            for orb_r in orbital_types:
                r = ao_index(atom(p), orb_r) # p & r on same atom
                chi_tensor[p,q,r] = chi_on_atom(orb(p), orb(q), orb(r), model_parameters)
    return chi_tensor

chi_tensor = calculate_chi_tensor(atomic_coordinates, model_parameters)
print(chi_tensor)

[[[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 4. 0. 0. 0. 0. 0. 0.]
  [0. 0. 4. 0. 0. 0. 0. 0.]
  [0. 0. 0. 4. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 4. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 4. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 4. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 

The multipole expansion plays the same role as an auxiliary basis set in modern resolution-of-identity (RI) methods that are used to accelerate large quantum chemistry simulations. The primary purpose of these methods is to decompose $V_{p,q,r,s}$ into a low-rank factored form, which is

$$ V_{p,q,r,s} = \sum_{t,u} \chi_{p,q,t} V_{t,u}^{\mathrm{ee}} \chi_{r,s,u} $$

in our semiempirical model. Unlike the previous vectors and matrices that we have constructed, the $\chi$ and $V$ tensors are sparse, meaning that most of their entries are zero. Without a more clever implementation, it can be computationally slow and wasteful to store and compute with these zero values. Modern quantum chemistry methods contain many ways of identifying and utilizing sparsity. For example, we could compute tensor elements of $\chi$ on-the-fly rather than storing the full tensor. This is analogous to "integral direct" methods in quantum chemistry. For simplicity, we will still precompute and store $\chi$, which fails to utilize its sparsity. However, we will avoid the explicit construction of $V_{p,q,r,s}$ by utilizing its factored form.

### E. 1-body Hamiltonian

The 1-body Hamiltonian coefficients $h_{p,q}$ combine many of the components that we have already discussed and implemented along with the on-site orbital energies, $E_s$ and $E_p$, that are the final two parameters from the semiempirical model,

$$ h_{p,q} = \begin{cases}
 t_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)})
  , & \mathrm{atom}(p) \neq \mathrm{atom}(q) \\
 E_{\mathrm{orb}(p)} \delta_{\mathrm{orb}(p),\mathrm{orb}(q)} + \sum_{r} \chi_{p,q,r} V_{r}^{\mathrm{ion}} , & \mathrm{atom}(p) = \mathrm{atom}(q) 
  \end{cases}. $$

In [57]:
def calculate_hamiltonian_matrix(atomic_coordinates, model_parameters):
    '''Returns the 1-body Hamiltonian matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    hamiltonian_matrix = np.zeros( (ndof,ndof) )
    potential_vector = calculate_potential_vector(atomic_coordinates, model_parameters)
    for p in range(ndof):
        for q in range(ndof):
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                hamiltonian_matrix[p,q] = hopping_energy(orb(p), orb(q), r_pq, model_parameters)
            if atom(p) == atom(q):
                if p == q and orb(p) == 's':
                    hamiltonian_matrix[p,q] += model_parameters['energy_s']
                if p == q and orb(p) in p_orbitals:
                    hamiltonian_matrix[p,q] += model_parameters['energy_p']
                for orb_r in orbital_types:
                    r = ao_index(atom(p), orb_r)
                    hamiltonian_matrix[p,q] += ( chi_on_atom(orb(p), orb(q), orb_r, model_parameters)
                                                 * potential_vector[r] )
    return hamiltonian_matrix

hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates, model_parameters)
print("hamiltonian matrix =\n",hamiltonian_matrix)

hamiltonian matrix =
 [[ 7.903767e-01  1.151972e+00  1.535963e+00  1.919954e+00 -7.357589e-04
   8.829107e-04  1.177214e-03  1.471518e-03]
 [ 1.151972e+00 -4.096233e-01  0.000000e+00  0.000000e+00 -8.829107e-04
  -4.031959e-03  2.472150e-03  3.090187e-03]
 [ 1.535963e+00  0.000000e+00 -4.096233e-01  0.000000e+00 -1.177214e-03
   2.472150e-03 -2.589871e-03  4.120250e-03]
 [ 1.919954e+00  0.000000e+00  0.000000e+00 -4.096233e-01 -1.471518e-03
   3.090187e-03  4.120250e-03 -7.357589e-04]
 [-7.357589e-04 -8.829107e-04 -1.177214e-03 -1.471518e-03  7.903767e-01
  -1.151972e+00 -1.535963e+00 -1.919954e+00]
 [ 8.829107e-04 -4.031959e-03  2.472150e-03  3.090187e-03 -1.151972e+00
  -4.096233e-01  0.000000e+00  0.000000e+00]
 [ 1.177214e-03  2.472150e-03 -2.589871e-03  4.120250e-03 -1.535963e+00
   0.000000e+00 -4.096233e-01  0.000000e+00]
 [ 1.471518e-03  3.090187e-03  4.120250e-03 -7.357589e-04 -1.919954e+00
   0.000000e+00  0.000000e+00 -4.096233e-01]]


We have now fully specified the many-body Hamiltonian $\hat{H}$, and we can now move on to approximating and calculating some of its physical properties.

## 3. Hartree-Fock theory

Even with a simple model for $\hat{H}$, we cannot calculate its ground state energy exactly for more than a few Argon atoms. Instead, we will use the [Hartree-Fock approximation](https://en.wikipedia.org/wiki/Hartree–Fock_method), which restricts the ground-state wavefunction to a single Slater determinant. We will find the Slater determinant with the lowest total energy, but a more general wavefunction will usually have an even lower energy. In the next section, we will use many-body perturbation theory to improve our estimate of the total energy.

The central objects of Hartree-Fock theory are the 1-electron density matrix $\rho_{p,q}$ and the Fock matrix $f_{p,q}$. These two matrices depend on each other, which defines a nonlinear set of equations that we must solve iteratively. Iteratively solving these equations is usually referred to as the self-consistent field (SCF) cycle. For this to converge, we must start from a reasonable initial guess for $\rho_{p,q}$. We will initialize it to the density matrix for isolated Argon atoms,

$$ \rho_{p,q}^{\mathrm{atom}} = \begin{cases} 
   1, & p = q \ \& \ \mathrm{orb}(p) \in \{ p_x, p_y, p_z \} \\
   0, & \mathrm{otherwise}
\end{cases} $$

In [18]:
orbital_occupation = { 's':0, 'px':1, 'py':1, 'pz':1 }

def calculate_atomic_density_matrix(atomic_coordinates):
    '''Returns a trial 1-electron density matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    density_matrix = np.zeros( (ndof,ndof) )
    for p in range(ndof):
        density_matrix[p,p] = orbital_occupation[orb(p)]
    return density_matrix

density_matrix = calculate_atomic_density_matrix(atomic_coordinates)
print(density_matrix)

print(2.0*np.einsum('pq,pq',hamiltonian_matrix,density_matrix))

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
-10.649737220952868


Because of spin symmetry, we use the same $\rho_{p,q}$ for each electron spin type, which reduces the sum over spin types in $\hat{H}$ to degeneracy pre-factors. Thus, half of the electrons are spin-up and half are spin-down.

The Fock matrix is defined by the density matrix,

$$ \begin{align} f_{p,q} & = h_{p,q} + \sum_{r,s} ( 2 V_{p,q,r,s} - V_{r,q,p,s} )\rho_{r,s} \\
                         & = h_{p,q} + \sum_{r,s,t,u} ( 2 \chi_{p,q,t} \chi_{r,s,u} - \chi_{r,q,t} \chi_{p,s,u} )
                             V_{t,u}^{\mathrm{ee}} \rho_{r,s}
   \end{align} $$

In [68]:
def calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor):
    '''Returns the Fock matrix defined by the input Hamiltonian, interaction, & density matrices.'''
    fock_matrix = hamiltonian_matrix.copy()
    fock_matrix += 2.0*np.einsum('pqt,rsu,tu,rs',
                                 chi_tensor, chi_tensor, interaction_matrix, density_matrix, optimize=True)
    fock_matrix -= np.einsum('rqt,psu,tu,rs',
                             chi_tensor, chi_tensor, interaction_matrix, density_matrix, optimize=True)
    return fock_matrix

fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
print(fock_matrix)

[[-1.197205e+01  3.279804e-01  4.373072e-01  5.466340e-01 -7.733021e-04
   9.045711e-04  1.206095e-03  1.507619e-03]
 [ 3.279804e-01 -4.920819e+00  1.187445e-01  1.484306e-01 -9.045711e-04
  -4.027730e-03  2.480541e-03  3.100676e-03]
 [ 4.373072e-01  1.187445e-01 -4.851551e+00  1.979075e-01 -1.206095e-03
   2.480541e-03 -2.580748e-03  4.134235e-03]
 [ 5.466340e-01  1.484306e-01  1.979075e-01 -4.762493e+00 -1.507619e-03
   3.100676e-03  4.134235e-03 -7.203426e-04]
 [-7.733021e-04 -9.045711e-04 -1.206095e-03 -1.507619e-03 -1.197205e+01
  -3.279804e-01 -4.373072e-01 -5.466340e-01]
 [ 9.045711e-04 -4.027730e-03  2.480541e-03  3.100676e-03 -3.279804e-01
  -4.920819e+00  1.187445e-01  1.484306e-01]
 [ 1.206095e-03  2.480541e-03 -2.580748e-03  4.134235e-03 -4.373072e-01
   1.187445e-01 -4.851551e+00  1.979075e-01]
 [ 1.507619e-03  3.100676e-03  4.134235e-03 -7.203426e-04 -5.466340e-01
   1.484306e-01  1.979075e-01 -4.762493e+00]]


which is just a sum of two tensor contractions between the $V$ tensor and the density matrix. Because the $V$ tensor has low rank, we can save a lot of computational effort by using its factored form. This is our first example of using "einsum" in NumPy to trivialize the implementation of a tensor equation.

The Fock matrix defines the matrix of molecular orbitals $\phi_{i,j}$ as its eigenvectors,

$$ \sum_{q} f_{p,q} \phi_{q,r} = E_r \phi_{p,r}, $$

and the density matrix is defined by the occupied orbitals (we occupy the lowest-energy orbitals with all of the electrons available),

$$ \rho_{p,q} = \sum_{i \in \mathrm{occ}} \phi_{p,i} \phi_{q,i}. $$

In [60]:
def calculate_density_matrix(fock_matrix):
    '''Returns the 1-electron density matrix defined by the input Fock matrix.'''
    num_occ = (ionic_charge//2)*np.size(fock_matrix,0)//orbitals_per_atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_matrix = orbital_matrix[:,:num_occ]
    density_matrix = occupied_matrix @ occupied_matrix.T
    return density_matrix

density_matrix = calculate_density_matrix(fock_matrix)
print(density_matrix)

[[ 9.933091e-01  3.458751e-02  4.611668e-02  5.764585e-02 -1.751243e-05
  -4.495922e-05 -5.994563e-05 -7.493204e-05]
 [ 3.458751e-02  8.212044e-01 -2.383942e-01 -2.979927e-01  4.495922e-05
  -3.152238e-06 -4.202984e-06 -5.253730e-06]
 [ 4.611668e-02 -2.383942e-01  6.821411e-01 -3.973237e-01  5.994563e-05
  -4.202984e-06 -5.603978e-06 -7.004973e-06]
 [ 5.764585e-02 -2.979927e-01 -3.973237e-01  5.033454e-01  7.493204e-05
  -5.253730e-06 -7.004973e-06 -8.756216e-06]
 [-1.751243e-05  4.495922e-05  5.994563e-05  7.493204e-05  9.933091e-01
  -3.458751e-02 -4.611668e-02 -5.764585e-02]
 [-4.495922e-05 -3.152238e-06 -4.202984e-06 -5.253730e-06 -3.458751e-02
   8.212044e-01 -2.383942e-01 -2.979927e-01]
 [-5.994563e-05 -4.202984e-06 -5.603978e-06 -7.004973e-06 -4.611668e-02
  -2.383942e-01  6.821411e-01 -3.973237e-01]
 [-7.493204e-05 -5.253730e-06 -7.004973e-06 -8.756216e-06 -5.764585e-02
  -2.979927e-01 -3.973237e-01  5.033454e-01]]


To calculate a $\rho_{p,q}$ and $f_{p,q}$ that are consistent with each other, we mix in a fraction of the new density matrix with the old density matrix and re-calculate them until convergence. This is a crude strategy for an SCF cycle, but it works for weakly interacting atoms like Argon:

In [62]:
def scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor):
    '''Returns converged density & Fock matrices defined by the input Hamiltonian, interaction, & density matrices.'''
    MAX_SCF_ITERATIONS = 100
    MIXING_FRACTION = 0.25
    CONVERGENCE_TOLERANCE = 1e-4
    for iteration in range(MAX_SCF_ITERATIONS):
        fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
        new_density_matrix = calculate_density_matrix(fock_matrix)

        error_norm = np.linalg.norm( density_matrix - new_density_matrix )
        if error_norm < CONVERGENCE_TOLERANCE:
            return density_matrix, fock_matrix

        density_matrix = (MIXING_FRACTION*new_density_matrix
                          + (1.0 - MIXING_FRACTION)*density_matrix)
    print("SCF cycle didn't converge")
    return density_matrix, fock_matrix

density_matrix, fock_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
print("density matrix after SCF:\n",density_matrix)

density matrix after SCF:
 [[ 9.895375e-01 -4.306584e-02 -5.742112e-02 -7.177640e-02  4.562670e-05
  -9.339187e-05 -1.245225e-04 -1.556531e-04]
 [-4.306584e-02  8.218833e-01 -2.374890e-01 -2.968612e-01  9.339187e-05
   8.212807e-06  1.095041e-05  1.368801e-05]
 [-5.742112e-02 -2.374890e-01  6.833480e-01 -3.958150e-01  1.245225e-04
   1.095041e-05  1.460055e-05  1.825068e-05]
 [-7.177640e-02 -2.968612e-01 -3.958150e-01  5.052313e-01  1.556531e-04
   1.368801e-05  1.825068e-05  2.281335e-05]
 [ 4.562670e-05  9.339187e-05  1.245225e-04  1.556531e-04  9.895375e-01
   4.306584e-02  5.742112e-02  7.177640e-02]
 [-9.339187e-05  8.212807e-06  1.095041e-05  1.368801e-05  4.306584e-02
   8.218833e-01 -2.374890e-01 -2.968612e-01]
 [-1.245225e-04  1.095041e-05  1.460055e-05  1.825068e-05  5.742112e-02
  -2.374890e-01  6.833480e-01 -3.958150e-01]
 [-1.556531e-04  1.368801e-05  1.825068e-05  2.281335e-05  7.177640e-02
  -2.968612e-01 -3.958150e-01  5.052313e-01]]


Once we have a converged density matrix, the Hartree-Fock total energy is defined as

$$ \begin{align}
  E_{\mathrm{HF}} &= E_{\mathrm{ion}} + E_{\mathrm{SCF}} \\
  E_{\mathrm{SCF}} &= 2 \sum_{p,q} h_{p,q} \rho_{p,q}
  + \sum_{p,q,r,s} V_{p,q,r,s} ( 2 \rho_{p,q} \rho_{r,s} - \rho_{p,s} \rho_{r,q} ) = \sum_{p,q} (h_{p,q} + f_{p,q}) \rho_{p,q}
  \end{align} $$

In [63]:
def calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix):
    '''Returns the Hartree-Fock total energy defined by the input Hamiltonian, Fock, & density matrices.'''
    energy_scf = np.einsum('pq,pq',hamiltonian_matrix + fock_matrix,density_matrix)
    return energy_scf

energy_scf = calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix)
print("Hartree-Fock energy =",energy_ion + energy_scf)

Hartree-Fock energy = -40.2355916899299


The bottleneck of this Hartree-Fock implementation is the tensor contraction in the formation of the Fock matrix. Because we are not exploiting the sparsity of $\chi$, it scales as $O(n^4)$ operations for $n$ atoms because of 3 free tensor indices and 1 summed tensor index in the contractions with the dense $\chi$ tensor.

## 4. 2nd-order Moller-Plesset (MP2) perturbation theory

The Hartree-Fock approximation does not describe the London dispersion interaction between Argon atoms. We must use many-body perturbation theory to improve our approximation to the ground state of $\hat{H}$. Thankfully, we only need to use the first correction beyond Hartree-Fock theory, which is [second-order Moller-Plesset (MP2) perturbation theory](https://en.wikipedia.org/wiki/Møller–Plesset_perturbation_theory). It is simple, but computationally expensive. The computational bottleneck is the transformation of the tensor $V_{p,q,r,s}$ from the atomic orbital basis to the molecular orbital basis,

$$ \tilde{V}_{a,i,b,j} = \sum_{p,q,r,s} \phi_{p,a} \phi_{q,i} \phi_{r,b} \phi_{s,j} V_{p,q,r,s}. $$

where we are restricting some indices to occupied orbitals ($i$ and $j$) and some indices to the unoccupied "virtual" orbitals ($a$ and $b$). This use of specific orbital labels to denote a restriction of indices is standard notation in quantum chemistry. To make this easier to program, we can partition the molecular orbitals and their energies into occupied and virtual:

In [23]:
def partition_orbitals(fock_matrix):
    '''Returns a list with the occupied/virtual energies & orbitals defined by the input Fock matrix.'''
    num_occ = (ionic_charge//2)*np.size(fock_matrix,0)//orbitals_per_atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_energy = orbital_energy[:num_occ]
    virtual_energy = orbital_energy[num_occ:]
    occupied_matrix = orbital_matrix[:,:num_occ]
    virtual_matrix = orbital_matrix[:,num_occ:]

    return occupied_energy, virtual_energy, occupied_matrix, virtual_matrix

occupied_energy, virtual_energy, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)

np.set_printoptions(precision=6)
print(f'occupied orbital energies:\n {occupied_energy}')
print("virtual orbital energies:\n", virtual_energy)

occupied orbital energies:
 [0.797491 0.797491 0.848047 0.848047 0.848047 0.848047]
virtual orbital energies:
 [2.104676 2.104676]


We use the factored form of $V_{p,q,r,s}$ to reduce our intermediate memory usage, but we do need to explicitly store $\tilde{V}_{a,i,b,j}$,

$$ \begin{align}
   \tilde{\chi}_{a,i,p} &= \sum_{q,r} \phi_{q,a} \phi_{r,i} \chi_{q,r,p} \\
   \tilde{V}_{a,i,b,j} &= \sum_{p,q} \tilde{\chi}_{a,i,p} V_{p,q}^{\mathrm{ee}} \tilde{\chi}_{b,j,q}
   \end{align} $$

In [70]:
def transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor):
    '''Returns a transformed V tensor defined by the input occupied, virtual, & interaction matrices.'''
    chi2_tensor = np.einsum('qa,ri,qrp', virtual_matrix, occupied_matrix, chi_tensor, optimize=True)
    interaction_tensor = np.einsum('aip,pq,bjq->aibj', chi2_tensor, interaction_matrix, chi2_tensor, optimize=True)
    return interaction_tensor

interaction_tensor = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor)
print(interaction_tensor)

[[[[ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00  6.606066e+00  0.000000e+00  0.000000e+00
    -4.983703e-15 -1.558558e-16]
   [-7.473911e-02  0.000000e+00  1.595946e-16  1.457168e-16
     0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
     0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00 -4.983703e-15  0.000000e+00  0.000000e+00
     7.634850e+00 -3.539910e-16]
   [-3.699270e-16  0.000000e+00 -2.666529e-02 -3.397458e-02
     0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00 -1.558558e-16  0.000000e+00  0.000000e+00


In this simple implementation, we are not taking full advantage of the sparisty of the initial $\chi$ tensor. Unlike the $V$ tensor, the $\tilde{V}$ tensor is constructed explicitly and stored in memory. The MP2 correction to the total energy is then just a large summation over previously computed and stored quantities,

$$ E_{\mathrm{MP2}} = - \sum_{a,b \in \mathrm{virt}} \sum_{i,j \in \mathrm{occ}} \frac{2 \tilde{V}_{a,i,b,j}^2 - \tilde{V}_{a,i,b,j} \tilde{V}_{a,j,b,i}}{E_a + E_b - E_i - E_j} . $$

In [71]:
def calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor):
    '''Returns the MP2 contribution to the total energy defined by the input Fock & interaction matrices.'''
    E_occ, E_virt, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)
    V_tilde = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor)

    energy_mp2 = 0.0
    num_occ = len(E_occ)
    num_virt = len(E_virt)
    for a in range(num_virt):
        for b in range(num_virt):
            for i in range(num_occ):
                for j in range(num_occ):
                    energy_mp2 -= ( (2.0*V_tilde[a,i,b,j]**2 - V_tilde[a,i,b,j]*V_tilde[a,j,b,i])
                                    /(E_virt[a] + E_virt[b] - E_occ[i] - E_occ[j]) )
    return energy_mp2

energy_mp2 = calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor)
print(energy_mp2)

-7.751533698349451


The bottleneck of MP2 calculations is the formation of $\tilde{V}$ from $\tilde{\chi}$ and $V^{\mathrm{ee}}$, which scales as $O(n^5)$ operations for $n$ atoms because of 4 free tensor indices and 1 summed tensor index once the contraction has been optimally arranged into intermediate operations.

## (Extra Credit) 5. Semiempirical model parameterization

For a semiempirical model to be useful, it must be parameterized to fit a set of reference data, usually from experiments or high-accuracy quantum chemistry simulation data. The parameters in this project have already been fit to data, which was carried out using the fitting procedure implemented in this section. Data fitting is an optimization problem, and we can make use of pre-existing Python optimization tools by encapsulating the fitting process into a function that inputs a list of parameters and outputs a fitting error that we want to minimize. We begin with our reference data, which is the set of occupied orbital energies, Hartree-Fock binding energies, and MP2 binding energies from all-electron calculations of Argon dimers at various separation distances:

In [32]:
# distance, HF binding energy, MP2 binding energy, 6 occupied HF orbital energies
reference_data = [ [ 5.67, 0.0064861, -0.0037137, -0.620241, -0.595697, -0.595697, -0.584812, -0.584812, -0.561277 ],
                   [ 6.24, 0.0022351, -0.0020972, -0.608293, -0.593456, -0.593456, -0.587742, -0.587742, -0.573604 ],
                   [ 6.99, 0.0005255, -0.0010120, -0.599442, -0.592082, -0.592082, -0.589651, -0.589651, -0.582576 ],
                   [ 7.18, 0.0003642, -0.0008494, -0.598041, -0.591886, -0.591886, -0.589922, -0.589922, -0.583988 ],
                   [ 10.21, 0.000000, -0.0000757, -0.591378, -0.591059, -0.591059, -0.590993, -0.590993, -0.590675 ] ]

We then write a function that encapsulates all of the setup and solving from earlier in the notebook to generate model predictions of the reference data:

In [75]:
def calculate_reference_data(atomic_coordinates, model_parameters):
    '''Returns the occupied orbital, HF, & MP2 energies for the input list of atomic coordinates'''
    ndof = len(atomic_coordinates)*orbitals_per_atom

    energy_ion = calculate_energy_ion(atomic_coordinates)
    density_matrix = calculate_atomic_density_matrix(atomic_coordinates)

    hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates, model_parameters)
    interaction_matrix = calculate_interaction_matrix(atomic_coordinates, model_parameters)
    chi_tensor = calculate_chi_tensor(atomic_coordinates, model_parameters)

    density_matrix, fock_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)

    occupied_energy, virtual_energy, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)
    energy_scf = calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix)
    energy_hf = energy_ion + energy_scf
    energy_mp2 = calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor)

    return occupied_energy, energy_hf, energy_mp2

print(calculate_reference_data(np.array([[0.0,0.0,0.0]]),model_parameters))
print(calculate_reference_data(np.array([[0.0,0.0,0.0],[12.0,0.0,0.0]]),model_parameters))
for r in np.arange(1.0,20.0,0.5):
    coord = np.array([[0.0,0.0,0.0],[r,0.0,0.0]])
    occupied_energy, energy_hf, energy_mp2 = calculate_reference_data(coord, model_parameters)
    print(r, energy_hf)
print(calculate_reference_data(atomic_coordinates, model_parameters))

(array([-12.200424,  -5.399548,  -5.399548]), -21.499174986562412, -4.383052748990632)
(array([-12.179337, -12.179305,  -5.375264,  -5.375264,  -5.374475,
        -5.374475]), -42.72059099032309, -8.748891929951823)
SCF cycle didn't converge
1.0 -31.444091250951757
SCF cycle didn't converge
1.5 -25.283666325836776
2.0 -29.30186030408747
2.5 -30.770913212508276
3.0 -32.302339947738105
3.5 -33.785877783038146
4.0 -35.15163427746512
4.5 -36.361561140160944
5.0 -37.403316593560646
5.5 -38.28374364651595
6.0 -39.02177032758299
6.5 -39.64164250144983
7.0 -40.16746576105844
7.5 -40.61964826016967
8.0 -41.0133270027311
8.5 -41.358426997627895
9.0 -41.660758928823874
9.5 -41.92352977126773
10.0 -42.14877107717178
10.5 -42.338394900014556
11.0 -42.49478663053626
11.5 -42.62098735666152
12.0 -42.72059099032309
12.5 -42.79749419047537
13.0 -42.85561241667808
13.5 -42.89863564547017
14.0 -42.929858535003554
14.5 -42.952090608931975
15.0 -42.96763455209573
15.5 -42.97831335185895
16.0 -42.9855266104

We can then write a function to compare the reference data to predictions for a given set of model parameters:

In [31]:
param_list = [ r_sk, t_sss, t_sps, t_pps, t_ppp, r0, r_pp, v_pp, energy_s, energy_p ]

def rms_model_error(param_list):
    


[5.0, 0, 0, 0, 0, 4.0, 5.0, 0.5, 1.3, 0.1]


Finally, we can use the SciPy optimizer to calculate the optimal model parameters:

## (Extra Credit) 6. Localized HF+MP2 calculations

Can I relate this section to SAPT?

An important area of ongoing research is the use of localization to accelerate QM calculations. Noble gases are an extreme example that enable localization to be applied very aggressively. If we will treat $t_{p,q}^{\mathrm{SK}}$ as a perturbation, then the Hartree-Fock equations are solved by our initial atomic guess for $\rho_{p,q}$. The energy contributions from pairs of atoms decays rapidly with distance, and we can ignore pairs of well-separated atoms.

In [27]:
max_distance = 15.0
nearby_pairs = [ (i,j) for i in range(num_atoms)
                       for j in range(num_atoms)
                       if i < j
                       and np.linalg.norm(coordinates[i]-coordinates[j]) <= max_distance ]
print(nearby_pairs)

NameError: name 'num_atoms' is not defined

This is a simple but inefficient method of forming the list of nearby atomic pairs.

The localized Hartree-Fock energy can be computed from the independent-atom solutions and a localized sum of pairwise atomic contributions that combine ion-ion, electron-ion, and electron-electron Coulomb energy contributions.

In [ ]:
v_ei_onsite = eval_kernel(v_ei_kernel)([0,0,0],[0,0,0])
v_ee_onsite = eval_kernel(v_ee_kernel)([0,0,0],[0,0,0])
energy_hf_local = num_atoms*Z*(E_P + v_ei_onsite + 0.5*Z*v_ee_onsite)

v_atom = eval_kernel(v_ii_kernel + Z*v_ei_kernel + (Z**2)*v_ee_kernel)
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    energy_hf_local += v_atom(r1,r2)

In addition to a localized version of the MP2 energy, we must also account for the perturbative correction to the Hartree-Fock energy upon introducing $t_{p,q}^{\mathrm{SK}}$,

$$ E_{\Delta HF} = - 2 \sum_{a \in \mathrm{virt}} \sum_{i \in \mathrm{occ}} \frac{(t_{a,i}^{\mathrm{SK}})^2}{E_a - E_i} . $$

The localized MP2 correction is identical to the standard correction, but the $\tilde{V}$ tensor can be replaced by the $V$ tensor and we can avoid storing and transforming an explicit tensor. In the atomic solution, the $p$ orbitals are occupied and the $s$ orbitals are virtual.

In [ ]:
energy_hf_delta = 0.0
for pair in nearby_pairs:
    r1 = coordinates[pair[0]]
    r2 = coordinates[pair[1]]
    for o in p_set:
        energy_hf_delta -= 2.0*(t_sk('s',o,r1-r2)**2 + t_sk(o,'s',r1-r2)**2) / (E_S - E_P)

energy_mp2_local = 0.0
for o1 in p_set:
    for o2 in p_set:
        a = index(0,'s')
        b = index(0,'s')
        i = index(0,o1)
        j = index(0,o2)
        energy_mp2_local -= num_atoms*((2.0*v_tensor(a,i,b,j)**2
                                        - v_tensor(a,i,b,j)*v_tensor(a,j,b,i))
                                        / (2.0*E_S - 2.0*E_P))
for pair in nearby_pairs:
    for o1 in p_set:
        for o2 in p_set:
            a = index(pair[0],'s')
            b = index(pair[1],'s')
            i = index(pair[0],o1)
            j = index(pair[1],o2)
            energy_mp2_local -= 4.0*v_tensor(a,i,b,j)**2 / (2.0*E_S - 2.0*E_P)

print(energy_hf_local,energy_hf_delta,energy_mp2_local)

Except for the formation of the nearby atomic pairs list, this entire localized calculation scales as $O(n)$ operations for $n$ atoms. This is an example of a QM calculation that has been stripped down to the point where it can be competitive in cost with simple interatomic potentials.